Scraping d'une page wikipédia en anglais, pour récupérer les caractéristiques suivantes :
  * Jour de sortie
  * Mois de sortie
  * Année de sortie
  * Durée
  * Pays
  * Budget min
  * Budget max
  * Box office min
  * Box office max

In [349]:
import bs4
import lxml
import pandas as pd
import urllib
import numpy as np
import math

from urllib import request

In [350]:
url = "https://en.wikipedia.org/wiki/Spirited_Away"

request_text = request.urlopen(url).read()
page = bs4.BeautifulSoup(request_text, "lxml")

In [351]:
table_info = page.find("table")

In [352]:
lignes = table_info.find_all("tr")

In [353]:
def get_date(date):
  """
  "(YYYY-MM-DD)" --> (YYYY, MM, DD)
  """
  date = date[1:-1] # enlever les paranthèses
  return date.split('-') # YYYY-MM-DD

In [354]:
def remove_notes(line):
  """
  Enlève une note entre crochets
  """
  n = len(line)
  i_start = -1
  i_end = -1
  for i in range(n):
    if line[i] == '[':
      i_start = i
    if line[i] == ']':
      i_end = i
      break
  if i_start == -1:
    return line
  return line[:i_start] + line[i+1:]

def remove_all_notes(line):
  """
  Enlève toutes les notes entre crochets
  """
  res = line
  prev_len_line = len(line)
  act_len_line = 0
  while prev_len_line != act_len_line:
    prev_len_line = act_len_line
    res = remove_notes(res)
    act_len_line = len(res)
  return res

In [355]:
def convert_money(liste_str):
  """
  "$XX.X XXXXXX" --> (nan, XXXXXXXX)
  ou
  "$XX.X-YY.Y XXXXXXXXXX" --> (XXXXXXXX, YYYYYYYYY)
  """
  values = liste_str[0][1:].split('–')
  if len(values) == 1:
    value_min = math.nan
  else:
    value_min = float(values[0])
  value_max = float(values[-1])
  match liste_str[1]:
    case "million" : return value_min * (10**6), value_max * (10**6)
    case "billion" : return value_min * (10**9), value_max * (10**9) # pour les pays anglophones, 1 billion est équivalent au 1 milliard en France
    case _ : return None

In [356]:
dict_res = {
    "Jour de sortie" : math.nan,
    "Mois de sortie" : math.nan,
    "Année de sortie" : math.nan,
    "Durée" : math.nan,
    "Pays" : math.nan,
    "Budget min" : math.nan,
    "Budget max" : math.nan,
    "Box office min" : math.nan,
    "Box office max" : math.nan
}
index_list = [0]

In [357]:
for ligne in lignes:
  ths = ligne.find_all("th")
  for th in ths:
    match th.text:
      case "Release date" :
        date = remove_all_notes(ligne.find("td").text).split()[3] # (YYYY-MM-DD)
        date = get_date(date) # YYYY, MM, DD
        dict_res["Année de sortie"] = date[0]
        dict_res["Mois de sortie"] = date[1]
        dict_res["Jour de sortie"] = date[2]
      case "Release dates":
        dates = remove_all_notes(ligne.find("td").text).split('\n') # sépare les lignes, enlève les notes entre crochets
        dates = [elem for elem in dates if elem != ''] # enlève les éventuels éléments vides
        date = dates[0].split() # s'il y a plusieurs dates, on prend seulement la 1ère
        date = get_date(date[3]) # (YYYY-MM-DD) --> YYYY, MM, DD
        dict_res["Année de sortie"] = date[0]
        dict_res["Mois de sortie"] = date[1]
        dict_res["Jour de sortie"] = date[2]
      case "Running time" :
        duree = remove_all_notes(ligne.find("td").text).split()[0] # durée (en minutes mais l'unité est toujours la même donc pas besoin de l'enregistrer)
        dict_res["Durée"] = duree
      case "Country" :
        pays = remove_all_notes(ligne.find("td").text)
        dict_res["Pays"] = pays
      case "Countries" :
        pays = remove_all_notes(ligne.find("td").text).split('\n') # sépare les lignes, enlève les notes entre crochets
        pays = [elem for elem in pays if elem != ''] # enlever les '' vides
        index_list = [i for i in range(len(pays))] # met à jour la liste d'index (pour le dataframe plus tard)
        dict_res["Pays"] = pays
      case "Budget" :
        budget = remove_all_notes(ligne.find("td").text).split()
        budget = convert_money(budget) # enlève le $, convertit million/billion dans la valeur
        dict_res["Budget min"] = budget[0]
        dict_res["Budget max"] = budget[1]
      case "Box office" :
        box_office = remove_all_notes(ligne.find("td").text).split()
        box_office = convert_money(box_office) # enlève le $, convertit million/billion dans la valeur
        dict_res["Box office min"] = box_office[0]
        dict_res["Box office max"] = box_office[1]

In [358]:
df = pd.DataFrame(dict_res, index=index_list) # obliger d'avoir index=[0,1] car sinon, il y a une erreur comme c'est une liste de pays de taille 2
print(df)

  Jour de sortie Mois de sortie Année de sortie Durée   Pays  Budget min  \
0             20             07            2001   125  Japan         NaN   

   Budget max  Box office min  Box office max  
0  19200000.0             NaN     395800000.0  
